# Obteniendo los datos estadisticos de todos los archivos

Se calculan todas las variables estadisticas y se almacenan en un dataframe

### Importando librerias

In [1]:
import pandas as pd
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import math
import numpy as np
import matplotlib.pyplot as plt

### Importar lista con nombres de archivos

*Cambiar el nombre de la lista según el sentimiento a analizar

In [2]:
files = pd.read_csv("Fear.csv") ### MODIFICAR EL NOMBRE

### Se crea un diccionario para almacenar las variables de cada archivo

In [3]:
data_instance = {'M1': [],
                 'M2': [],
                 'M3': [],
                 'M4': [],
                 'C2': [],
                 'C3': [],
                 'C4': [],
                 'Skewness': [],
                 'Kurtosis': [],
                 'Mean': [],
                 'Absolute_mean': [],
                 'Peak_value': [],
                 'SRV': [],
                 'RMS': [],
                 'Crest_factor': [],
                 'Shape_factor': []}

### Ciclo para leer todos los archivos de cada sentimiento

In [4]:
from statistics import mean


for a in range(int(files.count())):
    name_doc = files._get_value (a, 'Name')
    data = pd.read_csv (name_doc)

    ########################################################################################################################################
    ################# Fourier #################
    raw_data = np.ravel(data.to_numpy())
    n = len(raw_data)
    yf = np.fft.fft(raw_data)       #Get fourier transform
    yfts = np.fft.fftshift(yf)
    yftsa = 1.0/n * np.abs(yfts)        #Remove complex numbers

    # Convert numpy array to DataFrame
    data = pd.DataFrame (yftsa)
    ##########################################################################################################################################


    ################# Get statistical features #################
    m1 = float (stats.moment (data, moment = 1)) #First moment 
    m2 = float (stats.moment (data, moment = 2)) #Second moment
    m3 = float (stats.moment (data, moment = 3))#Third moment
    m4 = float (stats.moment (data, moment = 4)) #Fourth moment
    sc = float (m2 - math.sqrt (pow (m1,2))) #Second cumulant
    tc = float (m3 - (3*m1*m2) - (2*pow (m1,3))) #Third cumulant
    fc = float (m4 + (3*m3*m1) - (3*pow (m2,2)) + (12*m2*pow (m1,2)) - (6*pow (m1,4))) #Fourth cumulant
    skew = float (stats.skew (data)) #Skewness
    kurt = float (stats.kurtosis (data)) #Kurtosis
    mean = float(data.mean()) #Mean
    am = abs(float(data.mean())) #Absolute mean
    pv = 0.5*(float(data.max()) - float(data.min())) #Peak value
    da = abs(data)
    dsa = da**2
    srv = float(pow((dsa.sum()/int(dsa.count())),2)) #Square root value
    d2 = data**2
    rms = math.sqrt (float(d2.sum())/int(d2.count())) #Root mean square
    cf = pv/rms #Crest factor
    sf = rms/am #Shape factor

    ################# Append features to a dictionary #################
    data_instance ['M1'].append(m1)
    data_instance ['M2'].append(m2)
    data_instance ['M3'].append(m3)
    data_instance ['M4'].append(m4)
    data_instance ['C2'].append(sc)
    data_instance ['C3'].append(tc)
    data_instance ['C4'].append(fc)
    data_instance ['Skewness'].append(skew)
    data_instance ['Kurtosis'].append(kurt)
    data_instance ['Mean'].append(mean)
    data_instance ['Absolute_mean'].append(am)
    data_instance ['Peak_value'].append(pv)
    data_instance ['SRV'].append(srv)
    data_instance ['RMS' ].append(rms)
    data_instance ['Crest_factor'].append(cf)
    data_instance ['Shape_factor'].append(sf)

    del data

### Se crea un dataframe con todos los datos estadisticos y se guarda en un .csv

In [5]:
################# Create a dataframe from dictionary #################
data_stats = pd.DataFrame(data=data_instance)
data_stats.head()

,M1,M2,M3,M4,C2,C3,C4,Skewness,Kurtosis,Mean,Absolute_mean,Peak_value,SRV,RMS,Crest_factor,Shape_factor
0,0.0,3.227864e-07,4.063658e-09,8.527975e-11,3.227864e-07,4.063658e-09,8.496718e-11,22.158705,815.494108,0.000072,0.000072,0.021837,1.075763e-13,0.000573,38.130349,7.940494
1,0.0,1.931087e-07,1.612843e-09,2.689839e-11,1.931087e-07,1.612843e-09,2.678652e-11,19.005933,718.310746,0.000058,0.000058,0.019866,3.860905e-14,0.000443,44.816333,7.621042
2,0.0,2.792357e-07,2.200622e-09,3.603717e-11,2.792357e-07,2.200622e-09,3.580325e-11,14.913822,459.177372,0.000077,0.000077,0.019902,8.128944e-14,0.000534,37.272169,6.964973
3,0.0,2.842489e-07,2.941574e-09,4.779643e-11,2.842489e-07,2.941574e-09,4.755404e-11,19.410265,588.558540,0.000069,0.000069,0.019830,8.352895e-14,0.000538,36.887026,7.788188
4,0.0,3.095436e-07,3.034487e-09,4.526081e-11,3.095436e-07,3.034487e-09,4.497336e-11,17.619862,469.366023,0.000074,0.000074,0.019908,9.926542e-14,0.000561,35.467037,7.554555


In [6]:
#Normalizacion
scaler = MinMaxScaler()
data_stats = pd.DataFrame (scaler.fit_transform(data_stats),columns=['M1','M2','M3','M4','C2','C3','C4','Skewness','Kurtosis','Mean','Absolute_mean','Peak_value','SRV','RMS','Crest_factor','Shape_factor'])

data_stats.to_csv (r"..\stats\Normalizated_data\Stats_fear_fourier.csv")
data_stats.head()

,M1,M2,M3,M4,C2,C3,C4,Skewness,Kurtosis,Mean,Absolute_mean,Peak_value,SRV,RMS,Crest_factor,Shape_factor
0,0.0,0.300002,0.317631,0.413544,0.300002,0.317631,0.419075,0.221849,0.048141,0.356024,0.356024,1.000000,0.086708,0.534304,0.262127,0.827150
1,0.0,0.179348,0.126061,0.130437,0.179348,0.126061,0.132116,0.170719,0.041677,0.283891,0.283891,0.906669,0.031119,0.409433,0.326576,0.754958
2,0.0,0.259482,0.172005,0.174754,0.259482,0.172005,0.176589,0.104356,0.024441,0.379479,0.379479,0.908370,0.065521,0.496926,0.253855,0.606695
3,0.0,0.264146,0.229922,0.231777,0.264146,0.229922,0.234546,0.177277,0.033047,0.340023,0.340023,0.904991,0.067326,0.500438,0.250142,0.792731
4,0.0,0.287680,0.237185,0.219481,0.287680,0.237185,0.221818,0.148241,0.025119,0.367267,0.367267,0.908654,0.080009,0.523309,0.236454,0.739933
